<a href="https://colab.research.google.com/github/satanjj10/linebot_course/blob/main/%E5%9F%BA%E6%9C%AC%E7%B5%84%E6%88%90_%E7%AF%84%E6%9C%AC%E8%A8%8A%E6%81%AF%E7%9A%84%E7%BC%BA%E9%BB%9E_QuickReply.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
掛載Google硬碟
安裝套件
引用套件
APP應用準備
消息準備
handler執行方法設計
啟動應用

'''

'\n掛載Google硬碟\n安裝套件\n引用套件\nAPP應用準備\n消息準備\nhandler執行方法設計\n啟動應用\n\n'

In [ ]:
'''

資料 mapping 至google drive

把資料寫在/content/drive

即可保存在 google drive內

'''
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
'''

流程解析

'''


'\n\n流程解析\n\n'

In [ ]:
!pip install line-bot-sdk flask flask-ngrok

In [ ]:
'''
引用套件
'''

# 引用Web Server套件
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)


In [ ]:
'''
建置主程序

建置handler與 line_bot_api
'''

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
run_with_ngrok(app)

# 生成實體物件
line_bot_api = LineBotApi("lSRXXK98162m7Q+kBr64eBxc9tV9FaNDf5cUXtDF2M8ljxITO86UTkf0MyMROwaA5aU3NGhN7ic+DswXldXarwEhuFaJP2I/T+3/OEGYK6mPkXr9pi8kMKrq9gntscNDRQRe40ZjpoWM/UqW7l1LbAdB04t89/1O/w1cDnyilFU=")
handler = WebhookHandler("e3573ac18293efc5912dad6d88db1831")

In [ ]:
'''
建置主程序的API入口
  接受Line傳過來的消息
  並取出消息內容
  將消息內容存在google drive的檔案內
  並請handler 進行消息驗證與轉發
'''


# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print(body)

    # 記錄用戶log
    f = open("/content/drive/MyDrive/ai-event.log", "a")
    f.write(body)
    f.close()

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [ ]:
'''

準備QuickReply的Button

QuickReply快速回復

有很多小按鍵，按鍵使用後就會消失

準備QuickReply，必須先把按鍵都準備好


'''

# 引入相關套件
from linebot.models import (
    MessageAction, URIAction,
    PostbackAction, DatetimePickerAction,
    CameraAction, CameraRollAction, LocationAction,
    QuickReply, QuickReplyButton
)

# 創建QuickReplyButton 

## 點擊後，以用戶身份發送文字消息
## MessageAction
# QuickReplyButton
# action => 填Action
# MessageAction => 以用戶身分發文字
# label => 按鍵的文字
# text => 以用戶角度講的話
textQuickReplyButton = QuickReplyButton(
    action=MessageAction(
        label="發送文字消息", 
        text="text2"
    )
)

## 點擊後，彈跳出選擇時間之視窗
## DatetimePickerAction
# label => 按鍵的文字
# data => 傳隱藏資訊給我們
# mode date => 給用戶輸入的畫面是天數
dateQuickReplyButton = QuickReplyButton(
    action=DatetimePickerAction(
        label="時間選擇",
        data="data3",                       
        mode="date"
    )
)


## 點擊後，開啟相機
## CameraAction
cameraQuickReplyButton = QuickReplyButton(
    action=CameraAction(label="拍照")
)

## 點擊後，切換至照片相簿選擇
# 引導用戶發圖片消息給我們
cameraRollQRB = QuickReplyButton(
    action=CameraRollAction(label="選擇照片")
)

## 點擊後，跳出地理位置
# 引導用戶發地理位置消息給我們
locationQRB = QuickReplyButton(
    action=LocationAction(label="地理位置")
)

## 點擊後，以Postback事件回應Server
# 利用data做很多特殊功能
postbackQRB = QuickReplyButton(
    action=PostbackAction(label="我是PostbackEvent", data="Data1")
)

In [ ]:
'''

以QuickReply封裝該些QuickReply Button

'''
## 設計QuickReplyButton的List
# QuickReply的參數items通常為一個list
# list 放的是按鍵(button)
quickReplyList = QuickReply(
    items = [textQuickReplyButton, dateQuickReplyButton, cameraQuickReplyButton, cameraRollQRB, locationQRB,postbackQRB]
)

In [ ]:
'''

製作TextSendMessage，並將剛封裝的QuickReply放入

'''
## 將quickReplyList 塞入TextSendMessage 中 
from linebot.models import (
    TextSendMessage,
)
# QuickReply通常必須夾帶在xxxSendMessage的裡面
# xxxSendMessage都會有一個額外的quick_reply參數，此參數就是放QuickReply
quick_reply_text_send_message = TextSendMessage(text='發送問題給用戶，請用戶回答', quick_reply=quickReplyList)

In [ ]:
'''

設計一個字典
    當用戶輸入相應文字消息時，系統會從此挑揀消息

'''

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

# 根據自定義菜單四張故事線的圖，設定相對應image
# 待會用戶輸入@reply
template_message_dict = {
  "@reply":quick_reply_text_send_message,
}

In [ ]:
#用戶點擊button後，觸發postback event，對其回傳做相對應處理
from linebot.models import PostbackEvent

# PostbackEvent很重要，結合QuickReply可以做好玩的事情，不可逆的流程
@handler.add(PostbackEvent)
def handle_post_message(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)
    if (event.postback.data.find('Data1')== 0):
        with open("user_profile_business.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextSendMessage(
                    text='請稍待，會有專人與您聯繫'
                )
            )
    elif (event.postback.data.find('Data2') == 0):
        with open("user_profile_tutorial.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextSendMessage(
                    text='請稍待，我們會派專家您聯繫'
                )
            )
    else:
        pass

In [ ]:
# 用戶發出文字消息時， 按條件內容, 回傳文字消息
# 為了要觸發字典裡的消息，設計當用戶發文字，就會回字典裡找消息
# 記得傳@reply給機器人
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    
    if(event.message.text.find('@')!= -1):
        line_bot_api.reply_message(
        event.reply_token,
        template_message_dict.get(event.message.text)
        )
    else:
        line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text="字典內查無此字，請輸入@more")
        )

In [ ]:
# 主程序運行
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://3b8b-35-236-150-40.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [18/May/2022 07:00:48] "POST / HTTP/1.1" 200 -


{"destination":"U9b0c09d2905ecf335b8f6e4121a9c993","events":[]}


...

{"destination":"U9b0c09d2905ecf335b8f6e4121a9c993","events":[{"type":"postback","postback":{"data":"Data1"},"webhookEventId":"01G3AZJBNVKPJ30SS4Y0FT3RVY","deliveryContext":{"isRedelivery":false},"timestamp":1652857318616,"source":{"type":"user","userId":"Uf57398b2870127a0567cc1fc518d93e0"},"replyToken":"496ea7e7efb146a9a5017b4b939665ad","mode":"active"}]}


127.0.0.1 - - [18/May/2022 07:01:59] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2022 07:02:12] "POST / HTTP/1.1" 200 -


{"destination":"U9b0c09d2905ecf335b8f6e4121a9c993","events":[{"type":"message","message":{"type":"text","id":"16106794729214","text":"@reply"},"webhookEventId":"01G3AZJQKNRSD20175CBQEKR6B","deliveryContext":{"isRedelivery":false},"timestamp":1652857330809,"source":{"type":"user","userId":"Uf57398b2870127a0567cc1fc518d93e0"},"replyToken":"51cc6e5296d744b9942390194bbb3bb4","mode":"active"}]}


127.0.0.1 - - [18/May/2022 07:02:25] "POST / HTTP/1.1" 200 -


{"destination":"U9b0c09d2905ecf335b8f6e4121a9c993","events":[{"type":"message","message":{"type":"image","id":"16106795877901","contentProvider":{"type":"line"}},"webhookEventId":"01G3AZK59W74T1301WRAXEE23C","deliveryContext":{"isRedelivery":false},"timestamp":1652857345088,"source":{"type":"user","userId":"Uf57398b2870127a0567cc1fc518d93e0"},"replyToken":"c95700ee72844bf9984b99b9fbcff040","mode":"active"}]}
